In [1]:
import pandas as pd
import json
import os
import matplotlib.pyplot as plt

### Definition of input folder

In [2]:
folder_path = "../study"
stats_path = folder_path + "/stats"

In [19]:
df = pd.read_csv(folder_path + "/subjects/VNSLC_04/dMRI/microstructure/VNSLC_04_metrics.csv")
df.columns.size

1667

### Concatenation of all patient's metrics

In [29]:
## Read the list of subjects and for each subject do the tractography
dest_success = folder_path + "/subjects/subj_list.json"
with open(dest_success, 'r') as file:
    patient_list = json.load(file)
del file

dfs = []
for p_code in patient_list:
    metric_folder = "%s/subjects/%s/dMRI/microstructure/%s_metrics.csv" % (folder_path, p_code, p_code)
    if not os.path.exists(metric_folder):
        print(metric_folder, "doesn't exists")
        continue
    dfs.append(pd.read_csv(metric_folder))
    del metric_folder
del p_code

df = pd.concat(dfs, ignore_index=True, keys=["ID"], join="outer")
del dfs

print(df.columns.size)

info_df = pd.read_csv(stats_path + "/info.csv")

df = pd.merge(info_df, df, on="ID")
del info_df

if not os.path.isdir(stats_path):
    os.mkdir(stats_path)

df.to_csv("%s/dataset.csv" % stats_path, index=False)


1657


## Analysis

In [15]:
df = pd.read_csv("%s/dataset.csv" % stats_path)
df.head()

1861


AssertionError: 

### Division between features and label

In this first analysis we consider a binary classification problem. Therefore, we keep the NR-RP/R labels, in which the Partial responders are considered as Responders.

*In next analysis we will consider a multiclass classification problem, trying to classify all the three different types* 

In [5]:
y = df["NR-RP/R"]
X = df.drop(["NR-RP/R", "NR-RP-R"], axis=1) 

Here the explanation of some features:

NR-RP/R:

- NR-RP/R == 0 => Non Responder
- NR-RP/R == 1 => Partial responder or Responder

NR-RP-R:

- NR-RP-R == 0 => Non Responder
- NR-RP-R == 1 => Partial Responder
- NR-RP-R == 2 => Responder

epilepsy_type:

- epilepsy_type == 1 => Generalized
- epilepsy_type == 2 => Focal

In [15]:
X


,ID,age,therapy_duration,sex,AEDs,benzo,epilepsy_onset_age,epilepsy_type,epilepsy_duration,left-inf-longi-fasci_FA_mean,...,right-thalamus_frac_f1_skew,right-thalamus_frac_f1_kurt,left-thalamus_frac_csf_mf_mean,left-thalamus_frac_csf_mf_std,left-thalamus_frac_csf_mf_skew,left-thalamus_frac_csf_mf_kurt,right-thalamus_frac_csf_mf_mean,right-thalamus_frac_csf_mf_std,right-thalamus_frac_csf_mf_skew,right-thalamus_frac_csf_mf_kurt
0,VNSLC_01,36,93,1,3,0,5,1,31,0.157870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VNSLC_02,29,13,2,2,0,14,1,15,0.587306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VNSLC_03,65,234,1,3,0,15,1,50,0.519644,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VNSLC_04,34,11,2,2,0,8,2,26,0.489017,...,-0.000152,0.003746,0.097048,0.105738,0.003508,0.001913,0.112715,0.133663,0.006592,0.003719
4,VNSLC_06,62,136,2,2,0,36,2,26,0.550408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,VNSLC_07,46,216,2,2,0,5,2,41,0.567025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,VNSLC_09,47,13,2,3,0,2,2,45,0.581254,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,VNSLC_10,21,30,1,4,1,6,2,15,0.589103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,VNSLC_11,46,6,2,3,0,0,2,46,0.534865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,VNSLC_12,31,134,1,3,0,18,2,13,0.534950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
